In [3]:
import pandas as pd
import sqlalchemy
import requests
import re
from io   import StringIO
from bs4  import BeautifulSoup as BS
from time import sleep

In [20]:
import datetime as dt

In [40]:
import os

In [227]:
import warnings

## Custom Function Imports

In [76]:
from functions.logit import load_downloaded
from functions.status import print_refresh

In [237]:
from functions.df_func15 import low_price

In [208]:
## Important Constant Variables

In [212]:
# the Start and End Dates to look for Final Fantasy Prices
start_date = pd.to_datetime('2025-06-09', format='%Y-%m-%d')
end_date = pd.to_datetime('2025-06-15', format='%Y-%m-%d')

In [25]:
URL = 'https://www.mtggoldfish.com/tournament/pro-tour-final-fantasy#paper'

response = requests.get(URL)
response.status_code

sleep(30)


In [ ]:
'''
URL5 = 'https://www.mtggoldfish.com/tournament/pro-tour-final-fantasy?page=5'

response5 = requests.get(URL)
response5.status_code

sleep(30)
'''

In [ ]:
soup = BS(response.text)
print(soup.prettify())

In [ ]:
tournament = soup.find('table', attr={'class' : 'table-tournament'})

In [ ]:
print(soup.find('table', attrs={'class' : 'table-tournament'}).prettify())

In [ ]:
tournament_df = pd.read_html(
    StringIO(
        str(
            soup.find('table', attrs = {'class' : 'table-tournament'})
        )
    )
)[0]

In [ ]:
decks_table = soup.find('table')
deck_rows = decks_table.findAll('tr')
all_links = [deck.findAll('a') for deck in deck_rows[1:-1:2]] # Step by 2 because every other row was returning blank.
for link in all_links:
  print(link, '\n')
deck_links = [link[0].get('href') for link in all_links]
print(deck_links)
print(len(deck_links))
#deck_links = [deck.get('href') for row in
#for links in all_links[0::2]:
#    print(link, '\n')
#print(deck_links[0][0].get('href'))

In [21]:
tournament_df.head()

NameError: name 'tournament_df' is not defined

In [22]:
with open('../protour/final-fantasy/page1.htm') as f:
    soup_expanded = BS( f.read() )


In [23]:
print(
    soup_expanded.find('div', attrs={'class':'deck-table-container'} )  
)

<div class="deck-table-container">
<table class="deck-view-deck-table">
<tbody><tr class="deck-category-header">
<th colspan="4">
Creatures
(28)
</th>
</tr>
<tr>
<td class="text-right">
4
</td>
<td>
<span class="card_id card_name"><a data-card-id="Heartfire Hero [BLB]" data-full-image="https://cdn1.mtggoldfish.com/images/gf/Heartfire%2BHero%2B%255BBLB%255D.jpg" data-full-image-rotation="FALSE" data-original-title="" href="https://www.mtggoldfish.com/price/Bloomburrow/Heartfire+Hero#paper" rel="popover" title="">Heartfire Hero</a></span>
</td>
<td>
<span aria-label="mana cost: red" class="manacost"><i class="ms ms-r ms-cost ms-shadow"></i></span>
</td>
<td class="text-right">
$ 2.16
</td>
</tr>
<tr>
<td class="text-right">
4
</td>
<td>
<span class="card_id card_name"><a data-card-id="Hired Claw &lt;showcase&gt; [BLB]" data-full-image="https://cdn1.mtggoldfish.com/images/gf/Hired%2BClaw%2B%253Cshowcase%253E%2B%255BBLB%255D.jpg" data-full-image-rotation="FALSE" data-original-title="" href

In [24]:
deck_df = pd.read_html(
    StringIO(
        str(
            soup_expanded.find('div', attrs={'class':'deck-table-container'} )
        )
    )
)

In [19]:
deck_df[0]

,Creatures (28),Creatures (28).1,Creatures (28).2,Creatures (28).3
0,4,Heartfire Hero,NaN,$ 2.16
1,4,Hired Claw,NaN,$ 6.20
2,4,Emberheart Challenger,NaN,$ 6.80
3,3,Magebane Lizard,NaN,$ 3.00
4,4,Manifold Mouse,NaN,$ 5.92
5,4,Screaming Nemesis,NaN,$ 84.00
6,1,Tersa Lightshatter,NaN,$ 1.49
7,4,Twinmaw Stormbrood,NaN,$ 1.00
8,Spells (10),Spells (10),Spells (10),Spells (10)
9,4,Burst Lightning,NaN,$ 0.80


In [14]:
manifold_mouse = pd.read_csv(
    '../protour/final-fantasy/Manifold Mouse [BLB].csv',
    names = [ 'date' , 'price' ]
)

In [209]:
deck_df['manifold_mouse']['date']

TypeError: list indices must be integers or slices, not str

In [98]:
end_date = pd.to_datetime('2025-06-15', format='%Y-%m-%d')
start_date = pd.to_datetime('2025-06-09', format='%Y-%m-%d')

manifold_mouse['date'] = pd.to_datetime(manifold_mouse['date'])

temp = manifold_mouse.loc[
  manifold_mouse['date'].between(start_date, end_date, inclusive = 'both')                         
]

temp.nsmallest(1, 'price')

#for index, row in manifold_mouse.iterrows():
#  if start_date <= row['date'] and row['date'] <= end_date:
#    if minimum > row['price']:
#      minimum = row['price']
#    print(row['price'])

,date,price
336,2025-06-12,2.34


## Function Tests

In [175]:
test = low_price('Manifold Mouse', manifold_mouse, start_date, end_date)

C:\Users\School2\Documents\NSS-DA16\python\capstone\mtg-tournament-expenses\functions\df_func13.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices['card name'] = cardname


In [177]:
test.head()

,card name,low price
336,Manifold Mouse,2.34


In [131]:
test2 = test

In [145]:
test2.reset_index(drop = True, inplace = True)

In [148]:
test2['card name']='Manifold Mouse'
test2.set_index('card name').reset_index()

,card name,date,price,name
0,Manifold Mouse,2025-06-12,2.34,Manifold Mouse


In [ ]:
tes

In [146]:
test2.head()

,date,price,name,card name
0,2025-06-12,2.34,Manifold Mouse,Manifold Mouse


In [62]:
manifold_mouse['date']

0     2024-07-10
1     2024-07-11
2     2024-07-12
3     2024-07-13
4     2024-07-14
         ...    
396   2025-08-11
397   2025-08-12
398   2025-08-13
399   2025-08-14
400   2025-08-15
Name: date, Length: 401, dtype: datetime64[ns]

## CSV Price Dictionary

In [192]:
price_csv_dict = {}

csv_list = os.listdir('../cardprices')
for index, card_name in enumerate(card_list):
    card_list[index] = card_name.strip().strip("'").strip()
    
print(csv_list[0])
print(card_list[0])
# need to fix
card_list = load_downloaded()
for csv in csv_list:
  print_refresh(f'Looking for {csv} match: {len(price_csv_dict)} csv files found.              ')
  for card in card_list:
    if card.lower().strip().strip("'") == csv[:len( card.strip().strip("'") )].lower():
      price_csv_dict[card] = csv
      print_refresh(f'{csv} match found: {len(price_csv_dict)} csv files found.                     ')
      sleep(.01)
print( f'{ price_csv_dict[ card_list[0] ] }    ' )

csv_list[0]

Abhorrent Oculus [DSK].csv
Heartfire Hero
Heartfire Hero [BLB].csv    owcase_ [DMU] (F).csv match found: 389 csv files found.                      ::::::                  :::


'Abhorrent Oculus [DSK].csv'

In [193]:
price_csv_dict

{"'Abhorrent Oculus'\n": 'Abhorrent Oculus [DSK].csv',
 "'Abrade'\n": 'Abrade _borderless_ [INR].csv',
 "'Abuelo's Awakening'\n": "Abuelo's Awakening [LCI].csv",
 "'Aclazotz, Deepest Betrayal'\n": 'Aclazotz, Deepest Betrayal _showcase_ [LCI].csv',
 "'Adarkar Wastes'\n": 'Adarkar Wastes [EOC].csv',
 "'Agatha's Soul Cauldron'\n": "Agatha's Soul Cauldron [WOE].csv",
 "'Ambrosia Whiteheart'\n": 'Ambrosia Whiteheart [FIN].csv',
 "'Analyze the Pollen'\n": 'Analyze the Pollen [MKM].csv',
 "'Anoint with Affliction'\n": 'Anoint with Affliction [ONE].csv',
 "'Archfiend of the Dross'\n": 'Archfiend of the Dross _showcase_ [ONE].csv',
 "'Astrologian's Planisphere'\n": "Astrologian's Planisphere [FIN].csv",
 "'Atraxa, Grand Unifier'\n": 'Atraxa, Grand Unifier [ONE].csv',
 "'Authority of the Consuls'\n": 'Authority of the Consuls [FIC].csv',
 "'Aven Interrupter'\n": 'Aven Interrupter [OTJ].csv',
 "'Azure Beastbinder'\n": 'Azure Beastbinder [BLB].csv',
 "'Bandit's Talent'\n": "Bandit's Talent [BLB].c

In [204]:
test_string = f'../cardprices/{price_csv_dict[card_list[0]]}'
test_df = pd.read_csv(test_string)

In [203]:
test_df = pd.read_csv( '../cardprices/Torch the Tower [WOE].csv' )
test_df.head()

,2023-08-15,1.5
0,2023-08-16,1.36
1,2023-08-17,1.30
2,2023-08-18,1.24
3,2023-08-19,1.16
4,2023-08-20,1.13


In [205]:
test_df.head()

,2024-07-10,0.49
0,2024-07-11,3.0
1,2024-07-12,3.0
2,2024-07-13,3.0
3,2024-07-14,3.0
4,2024-07-15,3.0


In [179]:
low_price_df.head()

""


In [194]:
print( card_list[0:5] )
print( price_csv_dict[ card_list[0] ] )

["'Heartfire Hero'\n", "'Hired Claw'\n", "'Emberheart Challenger'\n", "'Magebane Lizard'\n", "'Manifold Mouse'\n"]
Heartfire Hero [BLB].csv


In [238]:
## Low Prices DataFrame

In [239]:
low_price_df = pd.DataFrame()

# warnings.filterwarnings('ignore')

for card_name, csv_file in price_csv_dict.items():
  csv_path = f'../cardprices/{csv_file}'
  prices_df   = pd.read_csv( csv_path, names = ['date', 'price'] )
  prices_df.head()
  card_row = low_price(card_name.strip().strip("'"), prices_df, start_date, end_date)
  low_price_df = pd.concat( [low_price_df, card_row] )

# warnings.resetwarnings()

,card name,low price
1017,"Zur, Eternal Schemer",6.94


In [242]:
low_price_df.reset_index(drop=True, inplace=True)

In [247]:
pd.options.display.max_rows = 500
low_price_df

,card name,low price
0,Abhorrent Oculus,17.07
1,Abrade,0.20
2,Abuelo's Awakening,0.30
3,"Aclazotz, Deepest Betrayal",4.27
4,Agatha's Soul Cauldron,28.62
5,Ambrosia Whiteheart,0.25
6,Analyze the Pollen,0.39
7,Anoint with Affliction,0.39
8,Archfiend of the Dross,0.62
9,Astrologian's Planisphere,2.71


In [249]:
len(card_list)

402

In [250]:
for card_name in card_list:
  if card_name not in price_csv_dict.keys():
    print(card_name)

'Screaming Nemesis'

'Twinmaw Stormbrood'

'Burst Lightning'

'Lightning Strike'

'Self-Destruct'

'Roaring Furnace // Steaming Sauna'

'Sidequest: Play Blitzball'

'Unholy Annex // Ritual Chamber'

'Defiled Crypt // Cadaver Lab'

'Summon: Bahamut'

'Summon: Knights of Round'

'Summon: Brynhildr'

'Summon: Fenrir'



In [206]:
help(low_price)

Help on function low_price in module functions.df_func13:

low_price(cardname: str, prices: pandas.core.frame.DataFrame, start, end) -> pandas.core.frame.DataFrame
    Takes a card name & price per day dateframe and returns the lowest price
    between the start and end dates as a one row dataframe.
    Date format '4-digit Year'-'2-digit month'-'2-digit day'.
    Requires date and price columns in dataframe.

